# TensorFlow Datasets

`tf.data` is a hihg-level API for reading data and transforming it into a form used for training. It is designed to work with TensorFlow and makes it easy to load data, manipulate it, and pipe it into a model. In short, It is a low-level API that can be used to build data pipelines. It should not be mistaken something to be used for loading only tensorflow datasets.

## 1. Imports and Configuration

In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_datasets as tfds

# Configure GPU memory growth to be dynamic instead of allocating all memory at once
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

/home/jay/anaconda3/envs/tf/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## 2. Data Loading and Preprocessing

In [2]:
(ds_train, ds_test), ds_info = tfds.load(
    "mnist",
    split=["train", "test"],
    shuffle_files=True,
    as_supervised=True,  # will return tuple (img, label) otherwise dict
    with_info=True,  # able to get info about dataset
)

Dl Completed...: 0 url [00:00, ? url/s]
Dl Completed...:  50%|█████     | 2/4 [00:00<00:01,  1.64 url/s]

Dl Completed...:  75%|███████▌  | 3/4 [00:00<00:00,  4.30 url/s]

Dl Completed...: 100%|██████████| 4/4 [00:01<00:00,  3.90 url/s]

Dl Completed...: 100%|██████████| 4/4 [00:01<00:00,  2.90 url/s]


Dataset mnist downloaded and prepared to /home/jay/tensorflow_datasets/mnist/3.0.1. Subsequent calls will reuse this data.


## 3. Model Definition

## 4. Compile Model

## 5. Model Training and Evaluation